<a href="https://colab.research.google.com/github/JimKing100/Jestimate/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import libraries
import pandas as pd
import numpy as np
import math

from datetime import datetime

In [326]:
# Load SF real estate data - 10 years (2009-2018) of single family home sales in San Francisco downloaded from the SF MLS
# Longitude and latitude were added to the csv file prior to loading using geocoding.geo.census.gov 
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Jestimate/master/data/SF-SFR-Sales-Final2d.csv')

# Rename subdistr_desc to neighborhood
df = df.rename(columns={'subdist_no': 'nid', 'subdist_desc': 'neighborhood'})

# Check the data
print(df.shape)
df.head(5)

(23711, 39)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,on_market_date,cdom,orig_list_price,sale_date,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,03/14/13,4,1095000,03/22/13,1260000,8,3.5,4,T,Per Tax Records,2691,NaN,0,1969,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",0,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN"
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,12/03/15,144,1250000,04/25/16,1075000,9,3.0,4,T,Per Tax Records,2437,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,05/18/17,36,1395000,08/17/17,1525000,9,5.0,5,D,Per Architect,2597,0.0689,3000,1947,RM1,RGLR,0,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,0,0
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,06/27/10,42,725000,08/20/10,715000,5,1.0,2,T,Per Tax Records,1312,NaN,0,1939,RH1,RGLR,"PVDW,PVSW",GARG,0,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK"
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,06/07/12,24,1595000,07/13/12,1595000,8,4.0,4,T,Per Tax Records,3307,NaN,0,1951,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",0,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK"


In [327]:
# Check for nulls
df.isnull().sum()

longitude            469
latitude             469
elevation            469
full_address           0
city                   0
state                  0
street_no              0
street_name            0
street_suffix       1613
zip                    0
area                   0
district_no            0
district_desc          0
nid                    0
neighborhood           0
on_market_date         0
cdom                   0
orig_list_price        0
sale_date              0
sale_price             0
rooms                  0
baths                  0
beds                   0
sf_source              0
sf_source_decs         0
sf                     0
lot_acres          13379
lot_sf                 0
year_built             0
zoning             10880
lot_desc               0
drive_side             0
parking                0
park_leased            0
num_parking            0
shopping               0
transportation         0
type                   0
views                  0
dtype: int64

In [328]:
# Check for zeros
(df == '0').sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


longitude              0
latitude               0
elevation              0
full_address           0
city                   0
state                  0
street_no              0
street_name            0
street_suffix          0
zip                    0
area                   0
district_no            0
district_desc          0
nid                    0
neighborhood           0
on_market_date         0
cdom                   0
orig_list_price        0
sale_date              0
sale_price             0
rooms                  0
baths                  0
beds                   0
sf_source              0
sf_source_decs         0
sf                     0
lot_acres              0
lot_sf                 0
year_built             0
zoning                 0
lot_desc           11198
drive_side         12825
parking             2483
park_leased        13222
num_parking            0
shopping           10597
transportation      8392
type                5686
views              10635
dtype: int64

In [329]:
# Baseline of sale_price
df['sale_price'].mean()

1371454.081312471

In [330]:
# Train, test split on date of 01/01/2018
df['sale_date'] = pd.to_datetime(df['sale_date'], infer_datetime_format=True)
df['year_sold'] = df['sale_date'].dt.year
df['month_sold'] = df['sale_date'].dt.month
df['day_sold'] = df['sale_date'].dt.day

df['on_market_date'] = pd.to_datetime(df['on_market_date'], infer_datetime_format=True)
df['year_on_market'] = df['on_market_date'].dt.year
df['month_on_market'] = df['on_market_date'].dt.month
df['day_on_market'] = df['on_market_date'].dt.day

cutoff = pd.to_datetime('2018-01-01')
train = df[df['sale_date'] < cutoff]
test  = df[df['sale_date'] >= cutoff]

print(train.shape)
print(test.shape)

(21487, 45)
(2224, 45)


In [331]:
# Wrangle the data for train and test
def engineer_features(X):
  
  # Impute mean for null long/lat/elev based on mean of of neighborhood
  def feature_calc(feature, nid, f_dict):
    if math.isnan(feature):
      if (nid) in f_dict:
        new_feature = f_dict[nid]
        return new_feature
    else:
      return feature
    
    return feature
  
  temp = X[~X['longitude'].isna()].groupby(['nid'])['longitude'].mean()
  long_dict = dict(temp)
  X['longitude'] = X.apply(lambda x: feature_calc(x['longitude'], x['nid'], long_dict), axis=1)
  
  temp = X[~X['latitude'].isna()].groupby(['nid'])['latitude'].mean()
  lat_dict = dict(temp)
  X['latitude'] = X.apply(lambda x: feature_calc(x['latitude'], x['nid'], lat_dict), axis=1)
  
  temp = X[~X['elevation'].isna()].groupby(['nid'])['elevation'].mean()
  elev_dict = dict(temp)
  X['elevation'] = X.apply(lambda x: feature_calc(x['elevation'], x['nid'], elev_dict), axis=1)
  
  # Fill rooms zero values by adding beds and baths
  def room_calc(rooms_val, beds_val, baths_val):
    if rooms_val == 0:
      total = beds_val + baths_val
    else:
      total = rooms_val
      
    return total
  
  X['rooms'] = X.apply(lambda x: room_calc(x['rooms'], x['beds'], x['baths']), axis=1)
  
  # Fill baths zero values by adding beds and baths
  X.loc[(X['baths'] == 0)] = 1
  
  # Fill house square foot zero values with the average house square footage by bedroom for all single family homes in SF
  averagesf_data = X.groupby('beds').sf.mean()

  # Use average sf by bedroom for each 0 value in each bedroom group up to 9 bedrooms
  for i in range(0, 9): 
    X.loc[(X['sf'] == 0) & (X['beds'] == i), 'sf'] = averagesf_data.loc[i]

  # Use 10,000sf for anything over 9 bedrooms
  X.loc[temp_data['sf'] == 0, 'sf'] = 10000
  X = X.astype({'sf': int})
  
  # Fill lot_sf zero values by using lot_acres to calc
  def lotsf_calc(lotsf_val, lotacres_val):
    if lotsf_val == 0:
      total = lotacres_val * 43560
    else:
      total = lotsf_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  
  # Fill lot_acres zero values by using lot_sf to calc
  def lotacres_calc(lotacres_val, lotsf_val):
    if lotacres_val == 0:
      total = lotsf_val / 43560
    else:
      total = lotacres_val
      
    return total
  
  X['lot_sf'] = X.apply(lambda x: lotsf_calc(x['lot_sf'], x['lot_acres']), axis=1)
  X['lot_acres'] = X.apply(lambda x: lotacres_calc(x['lot_acres'], x['lot_sf']), axis=1)
  
  # Features with numeric zero's that should be nan's
  cols_with_nzeros = ['lot_sf', 'lot_acres', 'year_built']
  for col in cols_with_nzeros:
    X[col] = X[col].replace(0, np.nan)
    
  # Features with numeric zero's that should be nan's
  cols_with_czeros = ['lot_desc', 'drive_side', 'parking', 'park_leased',
                      'shopping', 'transportation', 'type', 'views']
  for col in cols_with_czeros:
    X[col] = X[col].replace('0', np.nan)
  
  # Drop date columns as they were converted to day, month, year columns
  X = X.drop(columns='sale_date')
  X = X.drop(columns='on_market_date')
  
  return X

train = engineer_features(train)
test = engineer_features(test)

train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

,longitude,latitude,elevation,full_address,city,state,street_no,street_name,street_suffix,zip,area,district_no,district_desc,nid,neighborhood,cdom,orig_list_price,sale_price,rooms,baths,beds,sf_source,sf_source_decs,sf,lot_acres,lot_sf,year_built,zoning,lot_desc,drive_side,parking,park_leased,num_parking,shopping,transportation,type,views,year_sold,month_sold,day_sold,year_on_market,month_on_market,day_on_market
0,-122.50965,37.780280,200.83,"2645 El Camino Del Mar, San Francisco, CA 94121",San Francisco,CA,2645,El Camino Del Mar,NaN,94121,1050,1,SF District 1,1050,1 - Outer Richmond,4,1095000,1260000,8.0,3.5,4,T,Per Tax Records,2691,NaN,NaN,1969.0,RH2,"RGLR,FNCD","PVDW,PVSW","ATCH,GARG",NaN,2,4BLK,1BLK,3STR,"PNRM,OCEN,PARK,GRDN",2013,3,22,2013,3,14
1,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,144,1250000,1075000,9.0,3.0,4,T,Per Tax Records,2437,0.0689,3000.0,1947.0,RM1,RGLR,NaN,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,NaN,NaN,2016,4,25,2015,12,3
2,-122.50929,37.762608,23.21,"1278 La Playa St, San Francisco, CA 94122",San Francisco,CA,1278,La Playa,St,94122,2030,2,SF District 2,2030,2 - Outer Sunset,36,1395000,1525000,9.0,5.0,5,D,Per Architect,2597,0.0689,3000.0,1947.0,RM1,RGLR,NaN,"ATCH,GARG,ATDR,INAC",ONST,1,2BLK,1BLK,NaN,NaN,2017,8,17,2017,5,18
3,-122.50924,37.777330,189.11,"590 48th Ave, San Francisco, CA 94121",San Francisco,CA,590,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,42,725000,715000,5.0,1.0,2,T,Per Tax Records,1312,NaN,NaN,1939.0,RH1,RGLR,"PVDW,PVSW",GARG,NaN,2,3BLK,1BLK,"ATAC,2STR,FIXR","WATR,OCEN,PARK",2010,8,20,2010,6,27
4,-122.50919,37.776695,175.89,"618 48th Ave, San Francisco, CA 94121",San Francisco,CA,618,48th,Ave,94121,1050,1,SF District 1,1050,1 - Outer Richmond,24,1595000,1595000,8.0,4.0,4,T,Per Tax Records,3307,NaN,NaN,1951.0,NaN,RGLR,PVDW,"ATCH,GARG,ATDR,INAC",NaN,3,4BLK,2BLK,3STR,"PNRM,CTYL,OCEN,PARK",2012,7,13,2012,6,7
